# Práctica: El Problema del Bandido Multibrazo (K-Armed Bandit)

<a href="https://colab.research.google.com/github/kikaymusic/EscuderoRodriguezSanchez/blob/dev/k_brazos/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este cuaderno es el punto de entrada principal para la práctica. Desde aquí se configura el entorno de trabajo y se accede a los estudios detallados de cada algoritmo implementado.

In [ ]:
import sys
import os

def setup_workdirectory(
    repo_url="https://github.com/kikaymusic/EscuderoRodriguezSanchez.git",
    branch="dev",
    repo_name="EscuderoRodriguezSanchez",
    directoy_name="k_brazos"
):
    """
    Función para detectar si el código se está ejecutando en Google Colab o en local
    y configurar el entorno de trabajo en consecuencia.
    """
    # Detectamos si estamos en Google Colab
    in_colab = "google.colab" in sys.modules

    if in_colab:
        print("Entorno detectado: Google Colab")

        # Definimos la ruta del repositorio en Colab
        repo_path = f"/content/{repo_name}"

        # Borramos el repositorio si ya existe para evitar conflictos
        if os.path.exists(repo_path):
            os.system(f"rm -rf {repo_path}")

        # Clonamos la rama especificada del repositorio
        os.system(f"git clone -b {branch} --single-branch {repo_url}")

        # Cambiamos al directorio del repositorio
        os.system(f"%cd {repo_path}")

        # Añadimos el directorio especificado al path
        sys.path.append(f'/content/{repo_name}/{directoy_name}')

    else:
        print("Entorno detectado: Local")

        # Añadir el directorio especificado al path de Python
        notebook_dir = os.path.dirname(os.path.abspath('__file__')) if '__file__' in globals() else os.getcwd()
        k_brazos_dir = notebook_dir if directoy_name in notebook_dir else os.path.join(notebook_dir, directoy_name)
        if k_brazos_dir not in sys.path:
            sys.path.insert(0, k_brazos_dir)


setup_workdirectory()

Clonando el repositorio EleazarRodriguezSanchez...
Cloning into 'EleazarRodriguezSanchez'...
fatal: could not read Username for 'https://github.com': No such device or address
Añadido /content/EleazarRodriguezSanchez al path del sistema.
Entorno configurado correctamente.


## Introducción al Problema

El objetivo de esta práctica es analizar el comportamiento de distintos algoritmos de aprendizaje por refuerzo ante el dilema de **exploración vs explotación**. Se utilizan "brazos" con diferentes distribuciones de probabilidad (Normal, Binomial y Bernoulli) para evaluar la capacidad de cada agente para maximizar la recompensa acumulada.

Se han implementado tres estrategias principales basándose en la literatura clásica de Sutton y Barto:
1.  **Epsilon-Greedy**
2.  **UCB1**
3.  **Softmax**

## Navegación entre Estudios

Haz clic en los siguientes enlaces para abrir los cuadernos de análisis específicos en Google Colab:

* ### [1. Experimento Epsilon-Greedy](https://colab.research.google.com/github/kikaymusic/EscuderoRodriguezSanchez/blob/dev/k_brazos/bandido_epsilon_greedy.ipynb)
    * Análisis de la influencia del parámetro de exploración aleatoria.
* ### [2. Experimento UCB1](https://colab.research.google.com/github/kikaymusic/EscuderoRodriguezSanchez/blob/dev/k_brazos/bandido_ucb.ipynb)
    * Estudio del optimismo ante la incertidumbre mediante cotas de confianza.
* ### [3. Experimento Softmax](https://colab.research.google.com/github/kikaymusic/EscuderoRodriguezSanchez/blob/dev/k_brazos/bandido_softmax.ipynb)
    * Evaluación de la selección probabilística basada en la temperatura de Boltzmann.